#Sistema di raccomandazione
Il sistema implementa un approccio di raccomandazione one-time, evitando la costruzione di matrici di similarità NxN. Applica un filtro hard sulla categoria e combina una similarità basata su ingredienti con una funzione deterministica di brand e segmento, pesate rispettivamente 0.8 e 0.2.



In [1]:
import pandas as pd
import numpy as np



1) Importo il df e creo un nuovo df con le colonne d'interesse



In [2]:
df = pd.read_excel("brand_categoria.xlsx")

df = df[['code', 'product_name','brand_name', 'brand_segment','macro_category']].copy()
df['code'] = df['code'].astype(str)
df['product_name'] = df['product_name'].astype(str)
df['brand_name'] = df['brand_name'].astype(str)
df['brand_segment'] = df['brand_segment'].astype(str)
df['macro_category'] = df['macro_category'].astype(str)


df.head(20)



,code,product_name,brand_name,brand_segment,macro_category
0,88582000939,2,1,mass_market,Other
1,0650240025396,zan zusi b b flash 30,1,mass_market,Other
2,5013965698897,1,1,mass_market,Other
3,3560070084074,deo men marine,1 de carrefour,mass_market,Deodorants
4,6134598000044,savon liquide main life,100 da,mass_market,Hygiene
5,5943044010046,gerocossen,12,mass_market,Other
6,20546472,cien,12,mass_market,Other
7,3700216252688,glycerine vegetale,123gelules,mass_market,Other
8,8859423200212,habino,13000,mass_market,Other
9,6130460000747,nan,150da,mass_market,Other


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
df_ing = pd.read_excel("ingr_cat.xlsx")

df_ing['code'] = df_ing['code'].astype(str)
df_ing['product_name'] = df_ing['product_name'].astype(str)
df_ing['macro_category'] = df_ing['macro_category'].astype(str)
df_ing['inci_name'] = df_ing['inci_name'].astype(str)

df_finale = (
    df_ing
    .groupby(['code', 'product_name', 'macro_category'])['inci_name']
    .apply(lambda lst: ", ".join(sorted(set(lst))))
    .reset_index()
    .rename(columns={'inci_name': 'ingredients'}))

df_finale.head(20)


,code,product_name,macro_category,ingredients
0,1000034026006,mask skin technology,Masks,"Benzoate, Benzyl, Betaine, Camellia Sinensis L..."
1,10002315,muscle soak salt,Other,"Benzyl Salicylate, Ci 42090, Ci 77007, Fragran..."
2,10003787,szampon,Other,"Benzyl Alcohol, Carbomer, Citric Acid, Cocamid..."
3,1002003038089,Desodorante,Deodorants,"7801013 Vitoria, Piso 37 Y 38, Torre"
4,1004469044826,Clogate,Other,Flooride
5,10049013,Shampoo gegen Läuse,Hair_Care,"Benzoesure, Benzylalkohol, Cocamidopropylbetai..."
6,10058084,Nivea Hydratante express,Other,"Benzy Alcohol, Cetearyl Alcohol, Citronellol, ..."
7,1011104,palmers cocoa butter hemp oil,Oils,"1, 2-Hexanediol, Acer Saccharum Extract, Canna..."
8,1013002015568,Toallitas humedas,Other,"Citric Acid, Decyl Glucoside Copheryl Acetate,..."
9,10161555,Born lucky hell gel,Other,"Acrylates Copolymer, Aminomethyl Propanol, Eth..."


**2. Assegnazione pesi ai brand_segment.**

Il sistema di suggerimento tiene conto del segmento di mercato e pesa la similarità in base ad esso.
Prodotti dello stesso segmento avranno peso maggiore.

In [5]:
SEGMENT_SIMILARITY = {
    ('middle', 'middle'): 0.7,
    ('mass_market', 'mass_market'): 0.7,
    ('luxury', 'luxury'): 0.7,

    ('mass_market', 'middle'): 0.5,
    ('middle', 'mass_market'): 0.5,

    ('mass_market', 'luxury'): 0.3,
    ('luxury', 'mass_market'): 0.3,

    ('middle', 'luxury'): 0.5,
    ('luxury', 'middle'): 0.5
}


**3. Creazione dell'array di similarità 1xK.**

Per il brand non uso una cosine similarity, ma una funzione deterministica basata su regole di business che tengono conto sia dell’identità del brand sia del segmento di mercato.

Il sistema applica un hard filter sulla categoria per ridurre lo spazio di ricerca e garantire coerenza semantica, e successivamente calcola una similarità pesata basata su ingredienti e su regole di brand e segmento.



In [6]:
def brand_segment_similarity_1xK(
    query_product_id,
    df,
    category_col='macro_category',
    segment_col='brand_segment'
):
    query_product_id = str(query_product_id)

    if query_product_id not in df['code'].values:
        raise ValueError("Codice prodotto non trovato")

    query_row = df[df['code'] == query_product_id].iloc[0]
    query_category = query_row[category_col]
    query_brand = query_row['brand_name']
    query_segment = query_row[segment_col]

    # Filtro hard: stessa categoria -> ricerca tra prodotti della stessa categoria
    df_filtered = df[df[category_col] == query_category].copy()
    df_filtered = df_filtered[df_filtered['code'] != query_id]
    #filtro per evitare che restituisca lo stesso prodotto

    similarities = []

    for _, row in df_filtered.iterrows():
        if row['brand_name'] == query_brand:
            similarities.append(1.0)#se il prodotto appartiene allo stesso brand avrà similarità massima
        else:
            pair = (query_segment, row[segment_col]) #altrimenti considera il segmento di mercato
            similarities.append(SEGMENT_SIMILARITY.get(pair, 0.0))

    return np.array(similarities), df_filtered




4. Applicazione della funzione e creazione dell'aray one-time a partire dal prodotto


In [7]:
query_id = input('Inserisci il codice del prodotto: ')

brand_sim = brand_segment_similarity_1xK(query_id, df)
#non ottengo una matrice di similarità NXN, ma un array 1xK:
#cioè la similarità del prodotto query rispetto a tutti gli altri prodotti della stessa categoria



#brand_sim è una tupla: il primo elemento è l'array 1xK, il secondo elemento è il dataframe con i prodotti candidati
#brand_sim == (similarity_array, df_filtered).
#la tupla serve a sapere a quali prodotti mi riferisco
brand_sim_array, df_brand_candidates = brand_sim



Inserisci il codice del prodotto: 4088600303376


In [8]:
brand_sim_array[:10] #vedo i primi dieci valori di similarità

array([0.7, 0.7, 0.7, 0.5, 0.5, 0.5, 0.5, 0.5, 0.7, 0.5])

In [9]:
df_brand_candidates.head(10) #vedo i primi dieci candidati


,code,product_name,brand_name,brand_segment,macro_category
19,8436567350524,night recovery cream,226ers,mass_market,Face_Care
48,5397209636495,kitten face mask,7thheaven,mass_market,Face_Care
71,8809429951502,rice 72 serum,9wishes,mass_market,Face_Care
78,3282779285889,a derma creme mains au lait d avoine rhealba,a derma,middle,Face_Care
81,3282770014105,aderma dermalibour barrier creme reparatrice 50ml,a derma,middle,Face_Care
85,3282770202120,protect creme spf50,a derma,middle,Face_Care
89,3282770073706,epitheliale ah duo creme reparatrice quot bell...,a derma,middle,Face_Care
112,7610313021109,symphytum crema,a vogel,middle,Face_Care
117,1250000000001,sandalwood face pack,aarong,mass_market,Face_Care
125,3700562300019,la creme de jour,absolution,middle,Face_Care


5. Con questa funzione si trovano i prodotti più simili a un prodotto dato, guardando solo gli ingredienti. Prima limita il confronto ai prodotti della stessa categoria, così il confronto ha senso.
Poi confronta gli ingredienti del prodotto scelto con quelli degli altri e dice quanto si assomigliano, usando un punteggio numerico.

In [23]:
def ingredient_similarity_1xK_detailed(
    query_product_id,
    df_finale,
    X_ing,      # matrice bag-of-words
    category_col='macro_category'
):
    query_product_id = str(query_product_id)

    if query_product_id not in df_finale['code'].values:
        raise ValueError("Codice prodotto non trovato")

    query_row = df_finale[df_finale['code'] == query_product_id].iloc[0]
    query_category = query_row[category_col]

    # Filtro hard: stessa categoria
    df_filtered = df_finale[df_finale[category_col] == query_category].copy()
    df_filtered = df_filtered[df_filtered['code'] != query_product_id]

    print(f"Trovati {len(df_filtered)} prodotti nella stessa categoria")

    idx_query = df_finale.index[df_finale['code'] == query_product_id][0]

    # Vettore query
    vec_query = X_ing[idx_query]

    # Vettori filtrati
    indices_filtered = df_filtered.index
    X_filtered = X_ing[indices_filtered]

    # Similarità coseno 1xK
    similarities = cosine_similarity(vec_query, X_filtered)[0]

    return np.array(similarities), df_filtered



In [24]:

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

vectorizer = CountVectorizer(token_pattern=r'[^,]+')
X_ing = vectorizer.fit_transform(df_finale['ingredients'])



In [25]:
query_id = input('Inserisci il codice del prodotto: ')
ing_sim = ingredient_similarity_1xK_detailed(query_id, df_finale, X_ing)
ing_sim_array, df_ing_candidates = ing_sim



Inserisci il codice del prodotto: 4088600303376
Trovati 1668 prodotti nella stessa categoria


In [14]:
ing_sim_array[:10]

array([0.2236068 , 0.28284271, 0.        , 0.33333333, 0.3939193 ,
       0.16666667, 0.5       , 0.38575837, 0.22941573, 0.25      ])

In [15]:
df_ing_candidates.head(10)

,code,product_name,macro_category,ingredients
10,101627,"Hydrating face mist, hyaluronic",Face_Care,"Calendula Officinalis Flower Extract, Disodium..."
19,10181040702,Crema para el busto,Face_Care,"Benzyl Alcohol, Benzyl Benzoate, Butylene Glyc..."
43,10795847750078,Glamour Luxe LED face mask,Face_Care,"Platinum-Cured, Silicones"
44,10879574052,ANAI RUI Caffeine Bakuchiol Firming Eye Cream,Face_Care,"Bakuchiol, Butyrospermum Parkii Butter, Caffei..."
48,11033661,Ultra facial cream,Face_Care,"Acrylatesc10-30 Alkyl Acrylate Crosspolymer, B..."
95,1133479,moisturizer,Face_Care,"Apple Cider Vinegar, Betaine, Cetearyl Octanoa..."
96,11415567,Crème mains concentrée,Face_Care,"Benzyl Alcohol, Cetearyl Alcohol, Glycerin, Pa..."
120,12114,Crema Hidratante,Face_Care,"Acrylates Crosspolymer, Alpha-Isomethyl Ionone..."
122,12282,Totalist Crema Anti Arrugas Jalea Real,Face_Care,"Alcohol Denat, Alpha-Isomethyl Ionone, Benzyl ..."
151,13436094,Care Cream hand soap,Face_Care,"Benzyl Salicylate, Butylphenyl Methylpropional..."


**5. Indicizzazione su codice prodotto**

Costruzione nuovo DF: unione dei due dataframe sul codice prodotto al fine del calcolo della similarità finale.

In [16]:
df_ing = df_ing_candidates[['code']].copy()
df_ing['ing_sim'] = ing_sim_array

df_brand = df_brand_candidates[['code']].copy()
df_brand['brand_sim'] = brand_sim_array

#allineo la similarità
df_merged = df_ing.merge(
    df_brand,
    on='code',
    how='inner'
)


In [17]:
#check per verificare che l'allineamento è corretto
assert len(df_merged) > 0
assert df_merged['code'].is_unique


**6. Calcolo della similarità finale**

Gli ingredienti hanno peso maggiore nel determinare il prodotto più simile (0.8). I brand hanno peso minore (0.2).


In [18]:
df_merged['final_similarity'] = (
    0.8 * df_merged['ing_sim'] +
    0.2 * df_merged['brand_sim']
)


**6. Costruzione dataframe per la visualizzazione del prodotto più simile**


In [19]:
risultato = pd.DataFrame({
    'query_product_id': query_id,
    'recommended_product_id': df_merged['code'],
    'final_similarity_score': df_merged['final_similarity'],

})
#ordino i prodotti per suggerire il prodotto più simile
risultato = risultato.sort_values(
    by='final_similarity_score',
    ascending=False
).reset_index(drop=True)

risultato['rank'] = risultato.index + 1

# arricchimento informativo (nome, brand, segmento)
risultato = risultato.merge(
    df[['code', 'product_name', 'brand_name', 'brand_segment']],
    left_on='recommended_product_id',
    right_on='code',
    how='left'
).drop(columns='code')

risultato = risultato[
    [
        'rank',
        'query_product_id',
        'recommended_product_id',
        'product_name',
        'brand_name',
        'brand_segment',
        'final_similarity_score'
    ]
]

risultato.head(10)



,rank,query_product_id,recommended_product_id,product_name,brand_name,brand_segment,final_similarity_score
0,1,4088600303376,3253581471838,l occitane en provence almond delicious hand c...,l occitane en provence,mass_market,0.629898
1,2,4088600303376,3574660260038,handcreme creme mains,neutrogena,middle,0.611682
2,3,4088600303376,3574660258288,hand cream concentrated unscented,neutrogena,middle,0.611682
3,4,4088600303376,3574661287898,sofort einziehende handcreme,neutrogena,middle,0.554220
4,5,4088600303376,4005808484843,after sun hidratante,nivea,mass_market,0.551597
5,6,4088600303376,3574661016702,hand cream,neutrogena,middle,0.547214
6,7,4088600303376,3574661369426,creme mains hydratante concentree,neutrogena,middle,0.547214
7,8,4088600303376,70501872963,ultra gentle hydrating cleanser,neutrogena,middle,0.547214
8,9,4088600303376,14451171,nivea aloe vera hand cream,nivea,mass_market,0.544145
9,10,4088600303376,7702031169536,crema liquida avena,johnson s baby,mass_market,0.540000
